In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE229598"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE229598"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE229598.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE229598.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE229598.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE229598_family.soft.gz', 'GSE229598_series_matrix.txt.gz']
Identified SOFT files: ['GSE229598_family.soft.gz']
Identified matrix files: ['GSE229598_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression of retinoblatoma"
!Series_summary	"Retinoblastoma is the most frequent intraocular malignancy in children. Little is known on the molecular basis underlying the biological and clinical behavior of this cancer. Here, using gene expression profiles, we demonstrate the existence of two major retinoblastoma subtypes that can be divided into six subgroups."
!Series_overall_design	"Gene expression of 114 samples retinoblastomas tumor using the Hg-U133 Plus 2.0 Affymetrix array"

Sample Characteristics Dictionary:
{0: ['tissue: retinoblastoma'], 1: ['rb1 mutation: yes', 'rb1 mutation: no', 'rb1 mutation: NA'], 2: ['growth: exophytic', 'growth: mixed', 'growth: endophytic', 'growth: NA'], 3: ['chroid: none', 'chroid: minimal', 'chroid: exten

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The background information says "Gene expression of 114 samples retinoblastomas tumor using the Hg-U133 Plus 2.0 Affymetrix array"
# This indicates that this dataset contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Let's check each variable

# 2.1 Data Availability
# For the trait - Retinoblastoma
# The key 1 has 'rb1 mutation: yes', 'rb1 mutation: no', 'rb1 mutation: NA'
# RB1 mutation status is more informative for retinoblastoma study
trait_row = 1

# For age - Not available in the sample characteristics
age_row = None

# For gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# For trait (Retinoblastoma RB1 mutation status)
def convert_trait(value):
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        if value == "yes":
            return 1  # Has RB1 mutation
        elif value == "no":
            return 0  # Does not have RB1 mutation
    return None  # NA or other values

# Since age and gender data are not available, we don't need to define those conversion functions
convert_age = None
convert_gender = None

# 3. Save Metadata
# Determine if trait data is available based on trait_row
is_trait_available = trait_row is not None

# Save initial filtering info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Extract clinical features using the clinical_data from previous steps
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM7165988': [1.0], 'GSM7165989': [1.0], 'GSM7165990': [1.0], 'GSM7165991': [0.0], 'GSM7165992': [0.0], 'GSM7165993': [0.0], 'GSM7165994': [0.0], 'GSM7165995': [1.0], 'GSM7165996': [0.0], 'GSM7165997': [0.0], 'GSM7165998': [0.0], 'GSM7165999': [1.0], 'GSM7166000': [nan], 'GSM7166001': [0.0], 'GSM7166002': [0.0], 'GSM7166003': [0.0], 'GSM7166004': [0.0], 'GSM7166005': [nan], 'GSM7166006': [0.0], 'GSM7166007': [nan], 'GSM7166008': [1.0], 'GSM7166009': [0.0], 'GSM7166010': [0.0], 'GSM7166011': [0.0], 'GSM7166012': [0.0], 'GSM7166013': [nan], 'GSM7166014': [0.0], 'GSM7166015': [0.0], 'GSM7166016': [0.0], 'GSM7166017': [nan], 'GSM7166018': [0.0], 'GSM7166019': [0.0], 'GSM7166020': [nan], 'GSM7166021': [0.0], 'GSM7166022': [0.0], 'GSM7166023': [0.0], 'GSM7166024': [0.0], 'GSM7166025': [0.0], 'GSM7166026': [0.0], 'GSM7166027': [0.0], 'GSM7166028': [nan], 'GSM7166029': [0.0], 'GSM7166030': [0.0], 'GSM7166031': [0.0], 'GSM7166032': [nan], 'GSM7166033': [

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1053_at', '117_at', '121_at', '1255_g_at', '1316_at', '1320_at',
       '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at',
       '1552256_a_at', '1552257_a_at', '1552261_at', '1552263_at',
       '1552266_at', '1552269_at', '1552271_at', '1552274_at', '1552276_a_at'],
      dtype='object', name='ID')

Gene expression data shape: (17586, 114)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (with patterns like "1053_at", "117_at", etc.)
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)
# Therefore, they need to be mapped to human gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols
# From the gene annotation preview, we can see that:
# - 'ID' column contains identifiers that match those in the gene expression data (e.g., '1053_at')
# - 'Gene Symbol' column contains the gene symbols we want to map to

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping
print("Gene mapping preview:")
print(gene_mapping.head())
print(f"Total gene mappings: {len(gene_mapping)}")

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the gene-level data
print(f"\nOriginal gene expression data shape (probe-level): {gene_data.shape}")
print(f"Gene-level expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping preview:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Total gene mappings: 45782

Original gene expression data shape (probe-level): (16900, 114)
Gene-level expression data shape: (16900, 114)
First few gene symbols:
Index(['A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS', 'AACSP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data extracted and saved in Step 2
try:
    clinical_features = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_features.shape}")
    print(f"Clinical data columns: {clinical_features.columns.tolist()}")
    
    # Set up the clinical data for linking
    # The clinical data is likely not indexed properly, so let's ensure it has proper index
    if not clinical_features.index.name:
        # First attempt to see if there's a GSM column we can use as index
        gsm_cols = [col for col in clinical_features.columns if col.startswith('GSM')]
        if gsm_cols:
            clinical_features.set_index(gsm_cols[0], inplace=True)
        # Otherwise, set the first column as index if it exists
        elif len(clinical_features.columns) > 1:
            clinical_features.set_index(clinical_features.columns[0], inplace=True)
    
    # Now rename the column to match the trait variable
    if len(clinical_features.columns) >= 1:
        clinical_features.rename(columns={clinical_features.columns[0]: trait}, inplace=True)
    
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If clinical data file can't be loaded, create a placeholder
    sample_ids = normalized_gene_data.columns
    clinical_features = pd.DataFrame(index=sample_ids)
    clinical_features[trait] = float('nan')
    is_trait_available = False

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape before cleaning: {linked_data.shape}")

# 3. Handle missing values in the linked data
if is_trait_available and trait in linked_data.columns:
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after missing value handling: {linked_data.shape}")

    # 4. Determine whether trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    is_trait_biased = True
    print("Trait column not found in linked data, marking as biased.")

# 5. Conduct final validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available and trait in linked_data.columns,
    is_biased=is_trait_biased,
    df=linked_data,
    note="Dataset contains retinoblastoma tumor samples with RB1 mutation status information."
)

# 6. Save the linked data if it passed quality checks
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (16761, 114)
First few normalized gene symbols: ['A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE229598.csv
Loaded clinical data with shape: (1, 114)
Clinical data columns: ['GSM7165988', 'GSM7165989', 'GSM7165990', 'GSM7165991', 'GSM7165992', 'GSM7165993', 'GSM7165994', 'GSM7165995', 'GSM7165996', 'GSM7165997', 'GSM7165998', 'GSM7165999', 'GSM7166000', 'GSM7166001', 'GSM7166002', 'GSM7166003', 'GSM7166004', 'GSM7166005', 'GSM7166006', 'GSM7166007', 'GSM7166008', 'GSM7166009', 'GSM7166010', 'GSM7166011', 'GSM7166012', 'GSM7166013', 'GSM7166014', 'GSM7166015', 'GSM7166016', 'GSM7166017', 'GSM7166018', 'GSM7166019', 'GSM7166020', 'GSM7166021', 'GSM7166022', 'GSM7166023', 'GSM7166024', 'GSM7166025', 'GSM7166026', 'GSM7166027', 'GSM7166028', 'GSM7166029', 'GSM7166030', 'GSM7166031', 'GSM7166032', 'GSM7166033', 'GSM7166034', 'GSM7166035', 'GSM7166036', 'GSM7166037', 'GSM7166038', 'GSM7166039', 'GSM7166040', 'GSM7166041', 'GSM7166042', 'GSM7166043', 'GSM7166044', 'GSM7166045', 'GSM7166046', 'GSM7166047', '